In [ ]:
import os
import cv2
import numpy as np
from osgeo import gdal, ogr, osr
from multiprocessing import Pool
from shapely.geometry import Polygon
import math

# 配置路径
base_path = "H:\\Global_tree_cover"
years = ["2000extent", "2020extent"]
output_path = "H:\\Global_tree_cover\\output"

# 从raster获取像素尺寸
def get_pixel_size(raster_file):
    raster = gdal.Open(raster_file)
    geotransform = raster.GetGeoTransform()
    return geotransform[1], geotransform[5]  # x pixel size, y pixel size

# 从raster获取projection
def get_projection(raster_file):
    raster = gdal.Open(raster_file)
    return raster.GetProjection()

# 将经纬度差转换为米
def latlon_to_meters(lat_diff, lon_diff, lat):
    # 平均地球半径，单位：米
    R = 6371e3  

    # 把经纬度转换为弧度
    lat1 = math.radians(lat)
    lat2 = math.radians(lat + lat_diff)
    lon_diff = math.radians(lon_diff)

    # 应用haversine公式
    a = math.sin((lat2-lat1)/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(lon_diff/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

# 获取像素尺寸（米）
def get_pixel_size_meters(raster_file):
    x_pixel_size, y_pixel_size = get_pixel_size(raster_file)
    raster = gdal.Open(raster_file)
    lat = raster.GetGeoTransform()[3] # 获取纬度
    lat = lat + 5
    x_pixel_size_meters = latlon_to_meters(0, x_pixel_size, lat)
    y_pixel_size_meters = latlon_to_meters(y_pixel_size, 0, lat)
    return x_pixel_size_meters, y_pixel_size_meters

# 创建Shapefile来存储边缘
def create_shapefile(path, projection):
    driver = ogr.GetDriverByName("ESRI Shapefile")
    if os.path.exists(path):
        driver.DeleteDataSource(path)
    data_source = driver.CreateDataSource(path)
    srs = osr.SpatialReference()
    srs.ImportFromWkt(projection)
    layer = data_source.CreateLayer("forest_edge", srs, ogr.wkbLineString)
    return data_source, layer

# 处理每个tif文件
def process_tif(args):
    year, tif_file = args
    print(f"Processing {tif_file}...")

    # 加载数据
    raster = gdal.Open(tif_file)
    data = raster.ReadAsArray()

    # 计算像素尺寸
    x_pixel_size, y_pixel_size = get_pixel_size_meters(tif_file)
    pixel_area = abs(y_pixel_size) * abs(y_pixel_size)
    
    # 找到边缘并计算面积
    contours, _ = cv2.findContours(data, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    total_forest_area = np.count_nonzero(data) * pixel_area
    total_edge_length = 0

    # 创建输出文件
    output_folder = os.path.join(output_path, f"{year}_boundaries_results")
    os.makedirs(output_folder, exist_ok=True)
    shapefile_path = os.path.join(output_folder, os.path.basename(tif_file).replace(".tif", ".shp"))
    data_source, layer = create_shapefile(shapefile_path, get_projection(tif_file))

    # 获取GeoTransform，以将像素坐标转换为地理坐标
    gt = raster.GetGeoTransform()

    # 保存边缘到shapefile
    for cnt in contours:
        # 检查是否在边界上
        if cv2.pointPolygonTest(cnt, (0, 0), False) >= 0 or cv2.pointPolygonTest(cnt, (data.shape[1]-1, 0), False) >= 0 or \
           cv2.pointPolygonTest(cnt, (data.shape[1]-1, data.shape[0]-1), False) >= 0 or cv2.pointPolygonTest(cnt, (0, data.shape[0]-1), False) >= 0:
            continue
        
        line = ogr.Geometry(ogr.wkbLineString)
        for point in cnt:
            x_pixel_center = point[0][0] + 0.5  # adjust for the center of pixel
            y_pixel_center = point[0][1] + 0.5
            x_geo = gt[0] + x_pixel_center * gt[1] + y_pixel_center * gt[2]
            y_geo = gt[3] + x_pixel_center * gt[4] + y_pixel_center * gt[5]
            line.AddPoint(float(x_geo), float(y_geo))
            total_edge_length += x_pixel_size * x_pixel_size  # Accumulate edge length

        feature = ogr.Feature(layer.GetLayerDefn())
        feature.SetGeometry(line)
        layer.CreateFeature(feature)
        feature = None
    
    data_source = None
    return total_forest_area, total_edge_length

for year in years:
    tif_folder = os.path.join(base_path, year)
    tif_files = sorted([os.path.join(tif_folder, f) for f in os.listdir(tif_folder) if f.endswith(".tif")])

    for tif_file in tif_files:
        result = process_tif((year, tif_file))

        total_forest_area = result[0]
        total_edge_length = result[1]

        print(f"For year {year}, file {tif_file}:")
        print(f"Total forest area: {total_forest_area} square meters")
        print(f"Total forest edge length: {total_edge_length} meters")
        print("  ")

Processing H:\Global_tree_cover\2000extent\00N_000E.tif...
For year 2000extent, file H:\Global_tree_cover\2000extent\00N_000E.tif:
Total forest area: 20457990157.852856 square meters
Total forest edge length: 723779742.2312888 meters
  
Processing H:\Global_tree_cover\2000extent\00N_010E.tif...


In [ ]:
import os
import cv2
import numpy as np
from osgeo import gdal, ogr, osr
from multiprocessing import Pool
from shapely.geometry import Polygon
import math

# 配置路径
base_path = "H:\\Global_tree_cover"
years = ["2000extent", "2020extent"]
output_path = "H:\\Global_tree_cover\\output"

# 从raster获取像素尺寸
def get_pixel_size(raster_file):
    raster = gdal.Open(raster_file)
    geotransform = raster.GetGeoTransform()
    return geotransform[1], geotransform[5]  # x pixel size, y pixel size

# 从raster获取projection
def get_projection(raster_file):
    raster = gdal.Open(raster_file)
    return raster.GetProjection()

# 将经纬度差转换为米
def latlon_to_meters(lat_diff, lon_diff, lat):
    # 平均地球半径，单位：米
    R = 6371e3  

    # 把经纬度转换为弧度
    lat1 = math.radians(lat)
    lat2 = math.radians(lat + lat_diff)
    lon_diff = math.radians(lon_diff)

    # 应用haversine公式
    a = math.sin((lat2-lat1)/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(lon_diff/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    return R * c

# 获取像素尺寸（米）
def get_pixel_size_meters(raster_file):
    x_pixel_size, y_pixel_size = get_pixel_size(raster_file)
    raster = gdal.Open(raster_file)
    lat = raster.GetGeoTransform()[3] # 获取纬度
    lat = lat + 5
    x_pixel_size_meters = latlon_to_meters(0, x_pixel_size, lat)
    y_pixel_size_meters = latlon_to_meters(y_pixel_size, 0, lat)
    return x_pixel_size_meters, y_pixel_size_meters

# 创建Shapefile来存储边缘
def create_shapefile(path, projection):
    driver = ogr.GetDriverByName("ESRI Shapefile")
    if os.path.exists(path):
        driver.DeleteDataSource(path)
    data_source = driver.CreateDataSource(path)
    srs = osr.SpatialReference()
    srs.ImportFromWkt(projection)
    layer = data_source.CreateLayer("forest_edge", srs, ogr.wkbLineString)
    return data_source, layer

# 检查轮廓是否位于边缘附近
def is_near_edge(contour, shape, margin):
    x_min = contour[:, :, 0].min()
    x_max = contour[:, :, 0].max()
    y_min = contour[:, :, 1].min()
    y_max = contour[:, :, 1].max()

    return x_min <= margin or x_max >= shape[1] - margin or y_min <= margin or y_max >= shape[0] - margin

# 处理每个tif文件
def process_tif(args):
    year, tif_file = args
    print(f"Processing {tif_file}...")

    # 加载数据
    raster = gdal.Open(tif_file)
    data = raster.ReadAsArray()

    # 计算像素尺寸
    x_pixel_size, y_pixel_size = get_pixel_size_meters(tif_file)
    pixel_area = abs(y_pixel_size) * abs(y_pixel_size)
    
    # 找到边缘并计算面积
    contours, _ = cv2.findContours(data, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    total_forest_area = np.count_nonzero(data) * pixel_area
    total_edge_length = 0

    # 创建输出文件
    output_folder = os.path.join(output_path, f"{year}_boundaries_results")
    os.makedirs(output_folder, exist_ok=True)
    shapefile_path = os.path.join(output_folder, os.path.basename(tif_file).replace(".tif", ".shp"))
    data_source, layer = create_shapefile(shapefile_path, get_projection(tif_file))

    # 获取GeoTransform，以将像素坐标转换为地理坐标
    gt = raster.GetGeoTransform()

    # 保存边缘到shapefile
    for cnt in contours:
        if is_near_edge(cnt, data.shape, margin=1):  # Skip contours near the edge
            continue
        
        line = ogr.Geometry(ogr.wkbLineString)
        for point in cnt:
            x_pixel_center = point[0][0] + 0.5  # adjust for the center of pixel
            y_pixel_center = point[0][1] + 0.5
            x_geo = gt[0] + x_pixel_center * gt[1] + y_pixel_center * gt[2]
            y_geo = gt[3] + x_pixel_center * gt[4] + y_pixel_center * gt[5]
            line.AddPoint(float(x_geo), float(y_geo))
            total_edge_length += x_pixel_size * x_pixel_size  # Accumulate edge length

        feature = ogr.Feature(layer.GetLayerDefn())
        feature.SetGeometry(line)
        layer.CreateFeature(feature)
        feature = None
    
    data_source = None
    return total_forest_area, total_edge_length

for year in years:
    tif_folder = os.path.join(base_path, year)
    tif_files = sorted([os.path.join(tif_folder, f) for f in os.listdir(tif_folder) if f.endswith(".tif")])

    for tif_file in tif_files:
        result = process_tif((year, tif_file))

        total_forest_area = result[0]
        total_edge_length = result[1]

        print(f"For year {year}, file {tif_file}:")
        print(f"Total forest area: {total_forest_area} square meters")
        print(f"Total forest edge length: {total_edge_length} meters")
        print("  ")
